In [ ]:
import build_network__
net = build_network__.BuildModel(1,1,5)

In [ ]:
import tensorflow as tf
import os, sys, datetime

In [ ]:
# you need to know your dataset sizes to make the placeholders
dataset_h = 66
dataset_w = 200
dataset_c = 3 
dataset_classes = 5

In [ ]:
job_name = "AlphaGriffin_TF_Server"
known_machines = []
# this is another process... worker...
known_machines.append('localhost:2222')
# this is this process
known_machines.append('localhost:2223')
#known_machines.append( {'cool_name': ['sock_ip:port', 'localhost:2223']} )
known_machines_dict = { '{}'.format(job_name): [x for x in known_machines] }
len_machines = len(known_machines)
print("There are {} known machines".format(len_machines))
print(known_machines_dict)

In [ ]:
cluster = tf.train.ClusterSpec(known_machines_dict)

In [ ]:
# this is the HOME position... "you are here"
server = tf.train.Server(cluster,
                         job_name,
                         task_index=1) 

In [ ]:
# for each worker/server/w/e in the KNOWN CLUSTER... so job name would be known here.
#index = 0
def build_graphs(num_conv, num_fc, num_outputs):
    all_graphs = []
    for index in range(len_machines):
        current_context = tf.Graph()
        current_worker = "/job:{}/task:{}".format(job_name, index)
        print("With {} build graph {}".format(current_worker, index))
        with tf.device(tf.train.replica_device_setter(worker_device=current_worker,
                                                      cluster=cluster)):
            input_tensor, label_tensor, learn_rate = net.build_inputs(dataset_h, dataset_w, dataset_c, dataset_classes)
            print("# Built Input")
            # take the inputed variables and implement them
            # x_image = net.build_outputs(input_tensor,
            #                            num_conv,
            #                             num_fc,
            #                             num_outputs)
            # 
            # print("# Built Output")
            # x_image = net.BUILD_CONV_LAYERS()
            
        all_graphs.append(current_context)
    return all_graphs

In [ ]:
mastergraph = build_graphs(1,1,5)
print(mastergraph)